In [35]:
!pip install osmnx
!pip install matplotlib
!pip install geopandas
!pip install folium
!pip install scikit-learn

In [36]:
import folium
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
from folium.plugins import MarkerCluster
from shapely.geometry import MultiPoint, Polygon
from shapely.ops import unary_union
from sklearn.cluster import DBSCAN
import numpy as np

In [37]:
place_name = "Porto, Portugal"
G = ox.graph_from_place(place_name, network_type='all')
tags = {'highway': 'bus_stop'}
bus_stops = ox.geometries_from_place(place_name, tags=tags)
bus_stops_buffer = bus_stops.to_crs('EPSG:3395').buffer(100).to_crs('EPSG:4326')
intersections = ox.graph_to_gdfs(G, nodes=True, edges=False)
near_bus_stops = intersections[intersections.geometry.apply(lambda x: bus_stops_buffer.contains(x).any())]

# Your function to check for bike-friendly paths
def is_bike_friendly(edge):
    return (
        edge.get('bicycle', 'no') in ('yes', 'designated', 'permissive') or
        edge.get('highway') == 'cycleway' or
        'cycleway' in edge
    )

_, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)]

# Convert the bike-friendly edges to the same metric CRS that you used for bus_stops
bike_friendly_edges = bike_friendly_edges.to_crs('EPSG:3395')

# Create a buffer around the bike-friendly edges, using the same metric CRS
bike_friendly_buffer = bike_friendly_edges.buffer(20)

# Get back to the same CRS method for easy plotting.
bike_friendly_buffer = bike_friendly_buffer.to_crs('EPSG:4326')

# Filtering using the previous buffer for bus_stops and near_bus_stops.
bus_stops_near_bike_friendly = bus_stops[bus_stops.geometry.within(bike_friendly_buffer.unary_union)]
near_bus_stops_near_bike_friendly = near_bus_stops[near_bus_stops.geometry.within(bike_friendly_buffer.unary_union)]
number_of_danger_points = len(near_bus_stops_near_bike_friendly)


coords = np.array([[point.x, point.y] for point in near_bus_stops_near_bike_friendly.geometry])

# Using DBSCAN to cluster points; eps is the max distance between points in a cluster
epsilon = 0.0005  # Adjust this value based on your data's coordinate system
db = DBSCAN(eps=epsilon, min_samples=1).fit(coords)

# Grouping points by clusters
clustered = {}
for label, point in zip(db.labels_, near_bus_stops_near_bike_friendly.geometry):
    if label not in clustered:
        clustered[label] = []
    clustered[label].append(point)
# Creating the Folium map
m = folium.Map(location=[41.1579, -8.6291], zoom_start=12)

# Adding bike-friendly paths to the map
for _, edge in bike_friendly_edges.iterrows():
    # Certifique-se de que as coordenadas estão no formato correto para o Folium (lat, lon)
    line = [[point[1], point[0]] for point in edge.geometry.coords]
    folium.PolyLine(line, color="green", weight=2, opacity=1).add_to(m)

for cluster in clustered.values():
    if len(cluster) > 1:
        # If there's more than one point in a cluster, calculate the centroid
        centroid = MultiPoint(cluster).centroid
        folium.CircleMarker([centroid.y, centroid.x], radius=3, color="red", fill=True).add_to(m)
    else:
        # If it's a single point, add it as it is
        point = cluster[0]
        folium.CircleMarker([point.y, point.x], radius=3, color="red", fill=True).add_to(m)

#Display map
m


/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_21907/1728839577.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bus_stops = ox.geometries_from_place(place_name, tags=tags)
